In [1]:
import numpy as np
import pandas as pd
from fast_histogram import histogram1d
from scipy import spatial
from shapely.geometry import Polygon, Point, MultiPoint
import matplotlib.pyplot as plt
from tqdm import tqdm as tqdm
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [2]:
def corr(features, density, r_min, r_max, dr):
    radius = features.r.mean()
    N = features.x.count()
    dists, orders, N = dists_and_orders(features, r_max * radius)
    r_values = np.arange(r_min, r_max, dr) * radius

    
    g, bins = np.histogram(dists, bins=r_values)
    g6, bins = np.histogram(dists, bins=r_values, weights=orders)
    bin_centres = bins[1:] - (bins[1] - bins[0]) / 2
    divisor = 2 * np.pi * r_values[:-1] * (bins[1] - bins[0]) * density * N

    g = g / divisor
    g6 = g6 / divisor
    return bin_centres, g, g6

def dists_and_orders(f, t=1000):
    idx = get_idx(f, t)
    dists = get_dists(f, idx)
    orders = get_orders(f, idx)
    return dists.ravel(), orders.ravel(), np.sum(idx)


def get_idx(f, t):
    return f.edge_distance.values > t
#     return f.x.values > 0


def get_dists(f, idx):
    x = f[['x', 'y']].values
    return spatial.distance.cdist(x[idx, :], x)


def get_orders(f, idx):
    orders = make_complex(f)
    order_grid = make_order_grid(orders, idx)
    return np.abs(order_grid)


def make_order_grid(orders, idx):
    orders = orders.reshape(-1, 1)
    return orders[idx] @ np.conj(orders).transpose()


def make_complex(f):
    return f['hexatic_order'].values


def flat_array(x):
    return np.concatenate([item.ravel() for item in x])

In [3]:
def add_edge_distance(data):
    points = data[['x', 'y']].values
    hull = spatial.ConvexHull(points)
    hull_points = points[hull.vertices, :]
    polygon = Polygon(hull_points)
    multi_point = MultiPoint(points)
    dists = [polygon.exterior.distance(p) for p in multi_point.geoms]
    data['edge_distance'] = dists
    return data, hull.volume

# Test one dataset

In [4]:
file = "/media/data/Data/BallBearing/HIPS/PhaseDiagramsNewPlate/2,10mm/80%/480.hdf5"
data = pd.read_hdf(file)
# data = add_edge_distance(data)


frame0 = data.loc[0].copy()
N = len(frame0)
frame0['r'] = frame0['size']/2
frame0, area = add_edge_distance(frame0)
r, g, g6 = corr(frame0, N/area, 1, 400, 0.1)
diameter = r[np.argmax(g)]

### Average over frames

In [5]:
def apply(df):
    df, area = add_edge_distance(df)
    r, g, g6 = corr(df, N)

In [7]:
plt.subplot(1, 2, 1)
plt.loglog(r/diameter, g-1)
plt.xlim([0.4, max(r/diameter)])
plt.xlabel('r/d')
plt.ylabel('g(r)')
plt.subplot(1, 2, 2)
plt.loglog(r/diameter, g6/g)
plt.xlim([0.4, max(r/diameter)])
plt.xlabel('r/d')
plt.ylabel('$g_6(r)/g(r)$')

<ipython-input-7-f946e067a6da>:7: RuntimeWarning: invalid value encountered in true_divide
  plt.loglog(r/diameter, g6/g)


Text(0, 0.5, '$g_6(r)/g(r)$')

In [31]:
import filehandling
files = filehandling.get_directory_filenames("/media/data/Data/BallBearing/HIPS/PhaseDiagramsNewPlate/2,42mm/80%/*.hdf5")

In [32]:
files = files[0]
data = pd.read_hdf(file)
data['r'] = data['size']/2
frame0 = data.loc[0].copy()

In [33]:
frame0, area = add_edge_distance(frame0)
frame0

,y,x,mass,size,ecc,signal,raw_mass,ep,hexatic_order,number_of_neighbors,user_rad,torder,r,edge_distance
frame,,,,,,,,,,,,,,
0,36.945611,834.197674,913.338625,1.744781,0.037381,65.091650,3881.0,0.004874,0.224477+0.214356j,6.0,6,-0.867968-0.496619j,0.872391,0.000000
0,37.228107,1242.014124,970.208172,1.760361,0.145300,68.174939,4123.0,0.004587,0.443073+0.028229j,6.0,6,-0.380931+0.924604j,0.880180,0.000000
0,38.255732,632.194885,776.988748,1.787424,0.171200,55.499196,3518.0,0.005378,0.538336-0.091643j,5.0,6,0.493582+0.869699j,0.893712,0.000000
0,37.907566,693.870515,878.394687,1.740026,0.115020,63.721299,3940.0,0.004801,0.304430-0.062944j,5.0,6,0.045652+0.998957j,0.870013,0.051840
0,38.406284,1002.803178,948.625151,1.790791,0.102492,66.804588,3951.0,0.004787,0.253487+0.153372j,4.0,6,-0.574488-0.818513j,0.895396,1.343878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1948.070216,1511.090664,887.987140,1.744590,0.053943,64.406475,3479.0,0.005438,0.333928-0.112574j,6.0,6,0.390273+0.920699j,0.872295,0.850297
0,1947.658555,1608.124335,901.005470,1.850188,0.072792,58.239897,3814.0,0.004960,-0.064394+0.005785j,5.0,6,-0.851369-0.524567j,0.925094,0.196644
0,1948.473174,977.649404,919.505202,1.803396,0.059376,67.147176,3784.0,0.004999,-0.062360+0.026859j,6.0,6,-0.857864+0.513877j,0.901698,0.000000


In [34]:
r_min, r_max, dr = 2, 100, 0.1
radius = frame0.r.mean()
N = frame0.x.count()
dists, orders, N = dists_and_orders(frame0, 0)#r_max * radius * 3)
r_values = np.arange(r_min, r_max, dr) * radius
density = len(frame0)/area

In [102]:
N, len(dists), frame0.x.count()

(7562, 57418266, 7593)

In [103]:
g, bins = np.histogram(dists, bins=r_values)
g6, bins = np.histogram(dists, bins=r_values, weights=orders)
bin_centres = bins[1:] - (bins[1] - bins[0]) / 2
divisor = 2 * np.pi * r_values[:-1] * (bins[1] - bins[0]) * density * len(
    dists)

In [118]:
plt.plot(r_values[:-1], divisor/len(dists))
plt.xlabel('r [pixels]')
plt.ylabel('$N_{expect}$')

Text(0, 0.5, '$N_{expect}$')

In [116]:
plt.plot(r_values[:-1], g/divisor)
plt.xlabel('r [pixels]')
plt.ylabel('g(r)')

Text(0, 0.5, 'g(r)')

In [35]:
import filehandling
files = filehandling.get_directory_filenames("/media/data/Data/BallBearing/HIPS/PhaseDiagramsNewPlate/2,42mm/80%/*.hdf5")
figure_data = {}
for i, file in tqdm(enumerate(files)):
    duty = int(file[-8:-5])
    data = pd.read_hdf(file)
    data['r'] = data['size']/2
    frame0 =data.loc[0].copy()
    frame0, area = add_edge_distance(frame0)
    density = len(frame0) / area
    r, g, g6 = corr(frame0, density, 1, 300, 0.1)
    if i == 0:
        diameter = r[np.argmax(g)]
    
    figure_data[duty] = (r, g, g6, diameter)


94it [07:29,  4.78s/it]


In [38]:
for key, data in figure_data.items():
    if key % 10 == 0:
        plt.figure()
        plt.subplot(1, 2, 1)
        plt.loglog(data[0]/diameter, data[1]-1)
        plt.title(key)
        plt.subplot(1, 2, 2)
        plt.loglog(data[0]/diameter, data[2]/data[1])
        plt.subplot(1, 2, 1)
        plt.loglog(data[0]/diameter, (data[0]/diameter)**(-1/3))
        plt.axvline(diameter)
        plt.xlim([0.5, 10])
        plt.subplot(1, 2, 2)
        plt.loglog(data[0]/diameter, (data[0]/diameter)**(-1/4))
        plt.axvline(diameter)
        plt.xlim([0.5, 10])

<ipython-input-38-1d8e60c8f324>:8: RuntimeWarning: invalid value encountered in true_divide
  plt.loglog(data[0]/diameter, data[2]/data[1])
<ipython-input-38-1d8e60c8f324>:8: RuntimeWarning: invalid value encountered in true_divide
  plt.loglog(data[0]/diameter, data[2]/data[1])
<ipython-input-38-1d8e60c8f324>:8: RuntimeWarning: invalid value encountered in true_divide
  plt.loglog(data[0]/diameter, data[2]/data[1])
<ipython-input-38-1d8e60c8f324>:8: RuntimeWarning: invalid value encountered in true_divide
  plt.loglog(data[0]/diameter, data[2]/data[1])
<ipython-input-38-1d8e60c8f324>:8: RuntimeWarning: invalid value encountered in true_divide
  plt.loglog(data[0]/diameter, data[2]/data[1])
<ipython-input-38-1d8e60c8f324>:8: RuntimeWarning: invalid value encountered in true_divide
  plt.loglog(data[0]/diameter, data[2]/data[1])
<ipython-input-38-1d8e60c8f324>:8: RuntimeWarning: invalid value encountered in true_divide
  plt.loglog(data[0]/diameter, data[2]/data[1])
<ipython-input-38-1d

In [39]:
plt.close(
'all'

)


In [20]:
import pickle

In [271]:
with open(new_file, 'wb') as f:
    pickle.dump(figure_data, f)
    

# Plot

In [15]:
from matplotlib.widgets import Slider

In [18]:
direc1 = "/media/data/Data/BallBearing/HIPS/PhaseDiagramsNewPlate/2,15mm/85%"
new_file1 = direc1 + '/correlation_data.pkl'
direc2 = "/media/data/Data/BallBearing/HIPS/PhaseDiagramsNewPlate/2,25mm/85%"
new_file2 = direc2 + '/correlation_data.pkl'

In [21]:
with open(new_file1, 'rb') as f:
    figure_data_2_15 = pickle.load(f)

with open(new_file2, 'rb') as f:
    figure_data_2_25 = pickle.load(f)

In [24]:
fig, ax = plt.subplots(1, 2)
plt.subplots_adjust(bottom=0.25)
keys = list(figure_data_2_15.keys())
key = keys[0]
r, g, g6, diameter = figure_data_2_15[key]
r2, g2, g62, diameter2 = figure_data_2_25[key]
diameter = r[np.argmax(g)] / 2
plot1, = ax[0].semilogy(r*pix_2_mm, g-1)
plot1b, = ax[0].semilogy(r2*pix_2_mm, g2-1)
# ax[0].set_xlim([0.4, 5])
ax[0].set_xlabel('r [mm]')
ax[0].set_ylabel('g')
plot2, = ax[1].semilogy(r, g6/g)
plot2b, = ax[1].semilogy(r2, g62/g2)
# ax[1].set_xlim([0.4, 5])
ax[1].set_xlabel('r [mm]')
ax[1].set_ylabel('g6/g')


s_ax = plt.axes([0.25, 0.1, 0.65, 0.03])
slider = Slider(s_ax, 'Duty', min(keys), max(keys), valinit=key, valstep=1)

def update(val):
    duty = slider.val
    r, g, g6, diameter = figure_data_2_15[duty]
    r2, g2, g62, diameter2 = figure_data_2_25[duty]
    diameter = r[np.argmax(g)] / 2
    plot1.set_xdata(r*pix_2_mm)
    plot1.set_ydata(g)
    plot1b.set_xdata(r2*pix_2_mm)
    plot1b.set_ydata(g2)
    plot2.set_xdata(r*pix_2_mm)
    plot2.set_ydata(g6/g)
    plot2b.set_xdata(r2*pix_2_mm)
    plot2b.set_ydata(g62/g2)
slider.on_changed(update)



<ipython-input-24-1ec3e8f63c70>:13: RuntimeWarning: invalid value encountered in true_divide
  plot2, = ax[1].semilogy(r, g6/g)
<ipython-input-24-1ec3e8f63c70>:14: RuntimeWarning: invalid value encountered in true_divide
  plot2b, = ax[1].semilogy(r2, g62/g2)


0

<ipython-input-24-1ec3e8f63c70>:33: RuntimeWarning: invalid value encountered in true_divide
  plot2.set_ydata(g6/g)
<ipython-input-24-1ec3e8f63c70>:35: RuntimeWarning: invalid value encountered in true_divide
  plot2b.set_ydata(g62/g2)


# Test fake data

In [225]:
import trigrid

In [241]:
x, y = trigrid.grid(10, 100)
data = pd.DataFrame({'x': x, 'y': y, 'r': 5, 'hexatic_order': 1 + 0j})

In [242]:
data, area = add_edge_distance(data)

In [243]:
density = len(data)/area

In [250]:
r, g, g6 = corr(data, density, 2, 100, 0.1)

(40401,) (8543, 40401)
(345145743,) (345145743,) 8543


In [245]:
plt.loglog(r, g-1)

In [251]:
diameter = r[np.argmax(g)]/sqrt(3)

In [256]:
plt.loglog(r/diameter, g-1, '-o')
# for peak in peak_guesses:
#     plt.axvline(peak, c='r')

In [248]:
from math import sqrt
peak_guesses = [1, sqrt(3), 2, sqrt(7), 3]

# Get pixel to mm conversion

In [3]:
from labvision import images, video
file = "/media/data/Data/BallBearing/HIPS/PhaseDiagramsNewPlate/2,15mm/85%/451.mp4"
vid = video.ReadVideo(file)
frame = vid.read_frame()

In [4]:
images.display(frame)

[]

In [5]:
crop_result = images.crop_rectangle(frame)

In [10]:
bbox = crop_result.bbox

In [9]:
bbox

[[1324, 92], [2431, 2056]]

In [11]:
dx = bbox.xmax - bbox.xmin
dy = bbox.ymax - bbox.ymin

In [14]:
L = dy
pix_2_mm = 193.73 / L

In [25]:
L * pix_2_mm

193.73

In [27]:
dx, dy

(1107, 1964)

In [31]:
cropped = images.crop_and_mask(frame, crop_result.bbox, crop_result.mask)

In [ ]:
images.display(cropped)

In [30]:
diameter

21.011111733189455